https://mp.weixin.qq.com/s/GEgWHc8JngNsZYHUtBjHNg

https://discussion.datafountain.cn/questions/2413?shareFromPage=PAGE_QUESTION

In [2]:
import numpy as np
import pandas as pd

In [5]:
train_sales  = pd.read_csv('data_origin/train/train_sales_data.csv')

evaluation_public = pd.read_csv('data_origin/evaluation_public.csv')

In [6]:
def rmse(predict,real,xishu,month):
        rmse1 = (((predict - (real / month.map(lambda x:float(x))) ).map(lambda x:np.sqrt(x**2)))*xishu).mean()
        return rmse1

In [7]:
model_name = (evaluation_public[['adcode','model']].drop_duplicates()).reset_index()

In [8]:
all_result = None
for i in np.arange(0,len(model_name)):
   adcode_mid = model_name[model_name['index'] == i]['adcode'][i]
   model_mid = model_name[model_name['index'] == i]['model'][i]
   train_sales_mid = train_sales[(train_sales['adcode']==adcode_mid)&(train_sales['model']==model_mid)]
   train_sales_mid = train_sales_mid.sort_values(['regYear','regMonth'])
   train_sales_mid['xishu'] = np.arange(len(train_sales_mid)-1,-1,-1)
   train_sales_mid['xishu'] = train_sales_mid['xishu'].map(lambda x:1/(x+1))#map(lambda x:np.exp(-0.1*x))
   train_sales_month = train_sales_mid.groupby('regMonth',as_index=False)['salesVolume'].agg({'mean_sum':'mean'})
   train_sales_month['month'] = train_sales_month['mean_sum'] / train_sales_month['mean_sum'].max()
   train_sales_mid = pd.merge(train_sales_mid,train_sales_month[['regMonth','month']],on=['regMonth'],how='left')
   train_sales_mid_max = train_sales_mid['salesVolume'].max()
   train_sales_mid_min = train_sales_mid['salesVolume'].min()
   evaluation_public_mid = evaluation_public[(evaluation_public['adcode']==adcode_mid)&(evaluation_public['model']==model_mid)]
   evaluation_public_mid = pd.merge(evaluation_public_mid,train_sales_month[['regMonth','month']],on=['regMonth'],how='left')
   rmse_min = np.inf
   best_num = None
   for num in np.arange(train_sales_mid_min+0.3 ,train_sales_mid_max,0.3):
        rmse1 = rmse(num,train_sales_mid['salesVolume'],train_sales_mid['xishu'],train_sales_mid['month'])
        if rmse1 < rmse_min:
            rmse_min = rmse1
            best_num = num
   evaluation_public_mid['forecastVolum'] = best_num 
   evaluation_public_mid['forecastVolum'] = evaluation_public_mid['forecastVolum']*evaluation_public_mid['month']
   all_result = pd.concat([all_result,evaluation_public_mid],axis=0)

In [9]:
all_result = all_result.sort_values(['id'])   

In [10]:
dfr = all_result[['id', 'forecastVolum']].round().astype(int)

In [11]:
dfr.head()

,id,forecastVolum
0,1,287
0,2,416
0,3,205
0,4,380
0,5,500


In [12]:

dfr.to_csv('ds_ccf_car_sales.csv', index=False)

In [13]:
dfr.forecastVolum.mean()

500.7301136363636

In [14]:
dfr.shape

(5280, 2)